In [17]:
import contextily as cx
import pandas as pd
import geopandas

## Loading data

In [18]:
lor_gdf = geopandas.read_file("data/lor_friedrichshain_kreuzberg.geojson")
streets_processed_gdf = geopandas.read_file("data/extras/streets_processed.geojson")
drive_ways_gdf = geopandas.read_file("data/extras/drive_ways.geojson")
service_processed_gdf = geopandas.read_file("data/extras/service_processed.geojson")
estimated_parking_lanes_gdf = geopandas.read_file("data/parking/estimated_parking_lanes.geojson")
estimated_parking_spots_kfz_gdf = geopandas.read_file("data/parking/estimated_parking_spots_kfz.geojson")
demographics_age_df = pd.read_csv("data/demographics/demographics_age_of_inhabitants.csv", thousands=" ")
demographics_background_df = pd.read_csv("data/demographics/demographics_background_of_inhabitants.csv", thousands=" ", decimal=",")
registered_cars_gdf = geopandas.read_file("data/demographics/registered_cars.geojson")

## Merging some dataframes

In [19]:
demographics_df = pd.merge(demographics_age_df, demographics_background_df, on=("lor_number", "lor"))

## Visualizing data

In [20]:
def get_lor_map():
    return lor_gdf.explore(
        column="PLR_NAME",
        legend=True,
        legend_kwds={"colorbar": False},
        style_kwds={"opacity": 0.4, "fillOpacity": 0.4},
        name="Friedrichshain-Kreuzberg"
    )

In [21]:
def plot_geodataframe(gdf: geopandas.GeoDataFrame):
    ax = gdf.plot(figsize=(20, 20), alpha=0.5, edgecolor="k")
    cx.add_basemap(ax, crs=gdf.crs)

In [22]:
estimated_parking_spots_kfz_lor_gdf = estimated_parking_spots_kfz_gdf.sjoin(lor_gdf, how="inner")

In [23]:
parking_spot_count_per_lor = estimated_parking_spots_kfz_lor_gdf \
    .groupby("PLR_NAME") \
    .size() \
    .to_frame("parking_spot_count") \
    .reset_index()

In [24]:
parking_spot_count_per_lor.head(5)

,PLR_NAME,parking_spot_count
0,Am Berlin Museum,983
1,Andreasviertel,3542
2,Askanischer Platz,2297
3,Barnimkiez,1397
4,Boxhagener Platz,1663


In [25]:
estimated_parking_spots_kfz_old_lor_gdf = estimated_parking_spots_kfz_gdf \
    .sjoin(registered_cars_gdf, how="inner")

In [26]:
parking_spot_count_per_old_lor = estimated_parking_spots_kfz_old_lor_gdf \
    .groupby("lor") \
    .size() \
    .to_frame("parking_spot_count") \
    .reset_index()

In [27]:
parking_spot_count_per_old_lor.head(5)

,lor,parking_spot_count
0,Andreasviertel,3799
1,Askanischer Platz,2301
2,Barnimkiez,1397
3,Boxhagener Platz,3697
4,Chamissokiez,2928


In [28]:
registered_cars_parking_spot_count_gdf = registered_cars_gdf \
    .merge(parking_spot_count_per_old_lor, on="lor")

In [29]:
registered_cars_parking_spot_count_gdf["vehicles_per_parking"] = \
    registered_cars_parking_spot_count_gdf["vehicles_overall"] \
    / registered_cars_parking_spot_count_gdf["parking_spot_count"]

In [30]:
registered_cars_parking_spot_count_gdf["cars_per_parking"] = \
    registered_cars_parking_spot_count_gdf["cars_only"] \
    / registered_cars_parking_spot_count_gdf["parking_spot_count"]

In [31]:
registered_cars_parking_spot_count_gdf.explore(
    column="cars_per_parking",
)

In [32]:
registered_cars_parking_spot_count_gdf.explore(
    column="vehicles_per_parking"
)